In [ ]:
%pip install "calcbench-api-client[Pandas]"

In [1]:
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pyarrow as pa
import numpy as np
import calcbench as cb
from ipydatagrid import DataGrid
from qgridnext import show_grid
from scipy.stats import zscore

### File is the Calcbench Point-In-Time data, see https://github.com/calcbench/notebooks/blob/master/standardized_numeric_point_in_time.ipynb
### Talk to us@calcbench.com if you want a historical file.

In [2]:
d = pq.read_table("C:/Users/andre/Downloads/standardized_data_oct_23.parquet")

In [41]:
expenses = [
    "CostOfRevenue",
    "SGAExpense",
    "InterestExpense",
    "IncomeTaxes",
]

expr = pc.field("metric").isin(["Revenue", "NetIncome"] + expenses)

In [4]:
# Only dealing with Revenue for now
# expr = pc.field("metric") == "Revenue"

In [5]:
metric_data = d.filter(expr).to_pandas()

In [6]:
metric_data["value"] = metric_data["value"].astype(float)

In [ ]:
# Remove small companies

#revenue = revenue[revenue["value"] > 1e7]

In [7]:
# Quarterly only
metric_data = metric_data[~metric_data.index.get_level_values("fiscal_period").str.endswith("-0")]

## Add SIC Code meta-data

In [8]:
company_data = cb.companies(entire_universe=True)

In [20]:
merged = metric_data.reset_index().merge(
    company_data[["ticker", "sic_code"]], right_on=["ticker"], left_on=["ticker"]
)[["ticker", "period_end", "sic_code", "value", 'metric']]

##### Split SIC code into levels, GICs codes would probably be better but Calcbench does not have them
###### https://www.sec.gov/corpfin/division-of-corporation-finance-standard-industrial-classification-sic-code-list

In [22]:
merged["top_level_SIC_code"] = merged["sic_code"].floordiv(1000).astype("string")
merged["second_level_SIC_code"] = (
    (merged["sic_code"] % 1000).floordiv(100).astype("string")
)
merged["third_level_SIC_code"] = (
    (merged["sic_code"] % 100).floordiv(10).astype("string")
)
merged["fourth_level_SIC_code"] = (merged["sic_code"] % 10).astype("string")

In [23]:
merged["ds"] = (merged["period_end"] + pd.offsets.QuarterEnd()).dt.normalize()

In [24]:
merged

,ticker,period_end,sic_code,value,metric,top_level_SIC_code,second_level_SIC_code,third_level_SIC_code,fourth_level_SIC_code,ds
0,0000739708,2010-06-30,4832,6.009160e+08,CostOfRevenue,4,8,3,2,2010-09-30
1,0000739708,2010-09-30,4832,5.790980e+08,CostOfRevenue,4,8,3,2,2010-12-31
2,0000739708,2010-12-31,4832,2.491906e+09,CostOfRevenue,4,8,3,2,2011-03-31
3,0000739708,2011-03-31,4832,5.840690e+08,CostOfRevenue,4,8,3,2,2011-06-30
4,0000739708,2011-06-30,4832,6.423400e+08,CostOfRevenue,4,8,3,2,2011-09-30
...,...,...,...,...,...,...,...,...,...,...
2243274,talo,2023-06-30,1311,3.318200e+07,SGAExpense,1,3,1,1,2023-09-30
2243275,talo,2023-09-30,1311,2.488800e+07,SGAExpense,1,3,1,1,2023-12-31
2243276,talo,2023-12-31,1311,3.723600e+07,SGAExpense,1,3,1,1,2024-03-31
2243277,talo,2024-03-31,1311,6.984100e+07,SGAExpense,1,3,1,1,2024-06-30


In [25]:
merged = merged[
    ~merged.isnull().T.any()
]  # get rid of rows with any null values, the model builder does not like nulls

In [ ]:
# only get retail companies 
#merged = merged[merged['top_level_SIC_code'] == '5']

In [27]:
# Get first record, there are revisions in the Calcbench data set.
merged = merged.groupby(["ticker", "ds", "metric"]).first()

In [15]:
# only want companies for which we have more than a year of history
merged = merged.groupby("ticker").filter(lambda g: g.shape[0] > 4)

In [17]:
# throw out outliers
z_scores = merged.groupby(["ticker", "metric"])["value"].transform(lambda x: zscore(x))
merged = merged[z_scores < 4]

In [30]:
show_grid(merged)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [58]:
merged.reset_index().to_parquet("test_train_data.parquet", index=False)

In [142]:
accounts = merged.unstack('metric')

In [162]:
calculated_net_income = accounts["y"]["Revenue"] - accounts["y"][expenses].sum(axis=1)

In [163]:
other = (accounts["y"]["NetIncome"] - calculated_net_income) * -1

In [166]:
other

ticker      ds        
0000739708  2010-09-30    2.892369e+09
            2010-12-31    2.958616e+09
            2011-03-31             NaN
            2011-06-30    2.685973e+09
            2011-09-30    3.042578e+09
                              ...     
talo        2023-09-30    5.404040e+08
            2023-12-31    5.434460e+08
            2024-03-31    4.790570e+08
            2024-06-30    7.766620e+08
            2024-09-30    7.903400e+08
Length: 391011, dtype: float64

In [192]:
accounts

period_end                                         \
metric                CostOfRevenue IncomeTaxes InterestExpense  NetIncome   
ticker     ds                                                                
0000739708 2010-09-30    2010-06-30  2010-06-30      2010-06-30 2010-06-30   
           2010-12-31    2010-09-30  2010-09-30      2010-09-30 2010-09-30   
           2011-03-31    2010-12-31  2010-12-31      2010-12-31 2010-12-31   
           2011-06-30    2011-03-31  2011-03-31      2011-03-31 2011-03-31   
           2011-09-30    2011-06-30  2011-06-30      2011-06-30 2011-06-30   
...                             ...         ...             ...        ...   
talo       2023-09-30    2023-06-30  2023-06-30      2023-06-30 2023-06-30   
           2023-12-31    2023-09-30  2023-09-30      2023-09-30 2023-09-30   
           2024-03-31    2023-12-31  2023-12-31      2023-12-31 2023-12-31   
           2024-06-30    2024-03-31  2024-03-31      2024-03-31 2024-03-31   
           2024-09-30    2024-06-30  2024-06-30      2024-06-30 2024-06-30   

                                                 sic_code              \
metric                   Revenue SGAExpense CostOfRevenue IncomeTaxes   
ticker     ds                                                           
0000739708 2010-09-30 2010-06-30 2010-06-30          4832        4832   
           2010-12-31 2010-09-30 2010-09-30          4832        4832   
           2011-03-31        NaT        NaT          4832        4832   
           2011-06-30 2011-03-31 2011-03-31          4832        4832   
           2011-09-30 2011-06-30 2011-06-30          4832        4832   
...                          ...        ...           ...         ...   
talo       2023-09-30 2023-06-30 2023-06-30          1311        1311   
           2023-12-31 2023-09-30 2023-09-30          1311        1311   
           2024-03-31 2023-12-31 2023-12-31          1311        1311   
           2024-06-30 2024-03-31 2024-03-31          1311        1311   
           2024-09-30 2024-06-30 2024-06-30          1311        1311   

                                                                    \
metric                InterestExpense NetIncome Revenue SGAExpense   
ticker     ds                                                        
0000739708 2010-09-30            4832      4832    4832       4832   
           2010-12-31            4832      4832    4832       4832   
           2011-03-31            4832      4832    <NA>       <NA>   
           2011-06-30            4832      4832    4832       4832   
           2011-09-30            4832      4832    4832       4832   
...                               ...       ...     ...        ...   
talo       2023-09-30            1311      1311    1311       1311   
           2023-12-31            1311      1311    1311       1311   
           2024-03-31            1311      1311    1311       1311   
           2024-06-30            1311      1311    1311       1311   
           2024-09-30            1311      1311    1311       1311   

                                  y                                           \
metric                CostOfRevenue IncomeTaxes InterestExpense    NetIncome   
ticker     ds                                                                  
0000739708 2010-09-30 -6.009160e+08  37979000.0    -385579000.0  -77207000.0   
           2010-12-31 -5.790980e+08  20415000.0    -389197000.0 -150392000.0   
           2011-03-31 -2.491906e+09  81601000.0    -372770000.0  -55627000.0   
           2011-06-30 -5.840690e+08  92661000.0    -369666000.0 -131363000.0   
           2011-09-30 -6.423400e+08   9184000.0    -358950000.0  -37975000.0   
...                             ...         ...             ...          ...   
talo       2023-09-30 -1.011650e+08  -6892000.0     -45632000.0   13677000.0   
           2023-12-31 -1.035480e+08  15865000.0     -45637000.0   -2103000.0   
           2024-03-31 -1.035460e+08   5081000.0 

In [190]:
accounts.swaplevel(axis=1).loc[:,('CostOfRevenue')]

period_end  sic_code             y top_level_SIC_code  \
ticker     ds                                                                 
0000739708 2010-09-30 2010-06-30      4832 -6.009160e+08                  4   
           2010-12-31 2010-09-30      4832 -5.790980e+08                  4   
           2011-03-31 2010-12-31      4832 -2.491906e+09                  4   
           2011-06-30 2011-03-31      4832 -5.840690e+08                  4   
           2011-09-30 2011-06-30      4832 -6.423400e+08                  4   
...                          ...       ...           ...                ...   
talo       2023-09-30 2023-06-30      1311 -1.011650e+08                  1   
           2023-12-31 2023-09-30      1311 -1.035480e+08                  1   
           2024-03-31 2023-12-31      1311 -1.035460e+08                  1   
           2024-06-30 2024-03-31      1311 -1.351780e+08                  1   
           2024-09-30 2024-06-30      1311 -1.573100e+08                  1   

                      second_level_SIC_code third_level_SIC_code  \
ticker     ds                                                      
0000739708 2010-09-30                     8                    3   
           2010-12-31                     8                    3   
           2011-03-31                     8                    3   
           2011-06-30                     8                    3   
           2011-09-30                     8                    3   
...                                     ...                  ...   
talo       2023-09-30                     3                    1   
           2023-12-31                     3                    1   
           2024-03-31                     3                    1   
           2024-06-30                     3                    1   
           2024-09-30                     3                    1   

                      fourth_level_SIC_code  
ticker     ds                                
0000739708 2010-09-30                     2  
           2010-12-31                     2  
           2011-03-31                     2  
           2011-06-30                     2  
           2011-09-30                     2  
...                                     ...  
talo       2023-09-30                     1  
           2023-12-31                     1  
           2024-03-31                     1  
           2024-06-30                     1  
           2024-09-30                     1  

[391011 rows x 7 columns]

In [181]:
y

metric                 CostOfRevenue  IncomeTaxes  InterestExpense  \
ticker     ds                                                        
0000739708 2010-09-30  -6.009160e+08   37979000.0     -385579000.0   
           2010-12-31  -5.790980e+08   20415000.0     -389197000.0   
           2011-03-31  -2.491906e+09   81601000.0     -372770000.0   
           2011-06-30  -5.840690e+08   92661000.0     -369666000.0   
           2011-09-30  -6.423400e+08    9184000.0     -358950000.0   
...                              ...          ...              ...   
talo       2023-09-30  -1.011650e+08   -6892000.0      -45632000.0   
           2023-12-31  -1.035480e+08   15865000.0      -45637000.0   
           2024-03-31  -1.035460e+08    5081000.0      -44295000.0   
           2024-06-30  -1.351780e+08   21573000.0      -50845000.0   
           2024-09-30  -1.573100e+08     983000.0      -48982000.0   

metric                   NetIncome       Revenue   SGAExpense  
ticker     ds                                                  
0000739708 2010-09-30  -77207000.0  1.490009e+09 -376637000.0  
           2010-12-31 -150392000.0  1.477347e+09 -382997000.0  
           2011-03-31  -55627000.0           NaN          NaN  
           2011-06-30 -131363000.0  1.320826e+09 -372710000.0  
           2011-09-30  -37975000.0  1.604386e+09 -408111000.0  
...                            ...           ...          ...  
talo       2023-09-30   13677000.0  3.672100e+08  -33182000.0  
           2023-12-31   -2103000.0  3.831350e+08  -24888000.0  
           2024-03-31   85898000.0  3.849590e+08  -37236000.0  
           2024-06-30 -112439000.0  4.299320e+08  -69841000.0  
           2024-09-30   12381000.0  5.491650e+08  -48247000.0  

[391011 rows x 6 columns]

In [182]:
y.insert(0, "Other", other)

In [183]:
y

metric                        Other  CostOfRevenue  IncomeTaxes  \
ticker     ds                                                     
0000739708 2010-09-30  2.892369e+09  -6.009160e+08   37979000.0   
           2010-12-31  2.958616e+09  -5.790980e+08   20415000.0   
           2011-03-31           NaN  -2.491906e+09   81601000.0   
           2011-06-30  2.685973e+09  -5.840690e+08   92661000.0   
           2011-09-30  3.042578e+09  -6.423400e+08    9184000.0   
...                             ...            ...          ...   
talo       2023-09-30  5.404040e+08  -1.011650e+08   -6892000.0   
           2023-12-31  5.434460e+08  -1.035480e+08   15865000.0   
           2024-03-31  4.790570e+08  -1.035460e+08    5081000.0   
           2024-06-30  7.766620e+08  -1.351780e+08   21573000.0   
           2024-09-30  7.903400e+08  -1.573100e+08     983000.0   

metric                 InterestExpense    NetIncome       Revenue   SGAExpense  
ticker     ds                                                                   
0000739708 2010-09-30     -385579000.0  -77207000.0  1.490009e+09 -376637000.0  
           2010-12-31     -389197000.0 -150392000.0  1.477347e+09 -382997000.0  
           2011-03-31     -372770000.0  -55627000.0           NaN          NaN  
           2011-06-30     -369666000.0 -131363000.0  1.320826e+09 -372710000.0  
           2011-09-30     -358950000.0  -37975000.0  1.604386e+09 -408111000.0  
...                                ...          ...           ...          ...  
talo       2023-09-30      -45632000.0   13677000.0  3.672100e+08  -33182000.0  
           2023-12-31      -45637000.0   -2103000.0  3.831350e+08  -24888000.0  
           2024-03-31      -44295000.0   85898000.0  3.849590e+08  -37236000.0  
           2024-06-30      -50845000.0 -112439000.0  4.299320e+08  -69841000.0  
           2024-09-30      -48982000.0   12381000.0  5.491650e+08  -48247000.0  

[391011 rows x 7 columns]

In [148]:
accounts.loc[:, ("y", "metric")]

KeyError: ('y', 'metric')